In [2]:
import requests
from bs4 import BeautifulSoup
import os
import json
from tqdm import tqdm
import numpy as np

In [3]:
links_file_path = "saved_files/saved_links.p"
page_count_file_path = "saved_files/extra_pages.p"

output_dir_path = "saved_files"
os.makedirs(output_dir_path, exist_ok=True)

output_file_path = os.path.join(output_dir_path, "move_commentary.txt")


In [4]:
all_data = []

saved_links = np.load(links_file_path, allow_pickle=True)
extra_pages = np.load(page_count_file_path, allow_pickle=True)

print(len(saved_links))
print(saved_links[:10])
print(extra_pages[:10])

11600
['https://gameknot.com/annotation.pl/tiger-claws?gm=72088', 'https://gameknot.com/annotation.pl/challenge-from-alextdc?gm=72071', 'https://gameknot.com/annotation.pl/shrewsbury-v-darwinian-knights?gm=72046', 'https://gameknot.com/annotation.pl/o-c-rijs-25-oktober-2024?gm=72042', 'https://gameknot.com/annotation.pl/maddocks-a-v-shrewsbury-a?gm=72003', 'https://gameknot.com/annotation.pl/a-naj-idea?gm=71984', 'https://gameknot.com/annotation.pl/lets-play-chess?gm=71967', 'https://gameknot.com/annotation.pl/sour-tart?gm=71958', 'https://gameknot.com/annotation.pl/chess-game?gm=71957', 'https://gameknot.com/annotation.pl/the-torphy-fish-that-got-away?gm=71954']
[0, 0, 2, 0, 0, 0, 0, 4, 2, 2]


In [6]:
worked = 0
with open(output_file_path,'w') as f:
    for idx, link in enumerate(tqdm(saved_links[:10]), start=1):#Sample
        try:
            r = requests.get(link)
            
            if r.status_code == 200:
                soup = BeautifulSoup(r.content, 'html.parser')
                
                rows = soup.find_all('tr')
                
                for row in rows:
                    move_td = row.find('td', rowspan='2', style=lambda value: value and 'vertical-align: top' in value)
                    comment_td = row.find('td', style="vertical-align: top;")
                    
                    if move_td and comment_td:
                        move_text = move_td.get_text(separator=' ', strip=True)
                        move_text = ' '.join(move_text.split())
                        
                        comment_text = comment_td.get_text(separator=' ', strip=True)
                        comment_text = ' '.join(comment_text.split())

                        f.write(f"{move_text} <sep> {comment_text}\n")
                        worked += 1

            else:
                pass
                #print(f"Failed to fetch link {idx} with status code {r.status_code}")
        except Exception as e:
            pass
            #print(f"Error while processing link {idx}: {e}")

print(f"Scraped data for {worked} entries.")
print(f"All data saved in: {output_file_path}")

In [5]:
with open(output_file_path, 'r') as f:
    data = f.readlines()

data[:10]

['1. c4 c5 2. Nf3 Nf6 3. d4 cxd4 4. Nxd4 g6 <sep> Many 4th moves for Black have been tried here, such as ... P - Q 4, ... Kt - B 3 and ... P - K 3. The line has not been fully worked out.\n',
 '5. Nc3 d5 <sep> White was threatening 6 P - K 4, transposing to the Maroczy bind.\n',
 '6. Bg5 <sep> A better choice than 6 P x P, which leads to an equal position.\n',
 '6... dxc4 7. e3 Qa5 <sep> Premature. Better was 7... B - Kt 2.\n',
 '8. Bxf6 exf6 9. Bxc4 Bb4 <sep> That he has played for the fianchetto and then developed the Bishop here is one reason Black is behind in development.\n',
 '10. Rc1 a6 <sep> Another time-wasting move. Better would be 10 O-O.\n',
 '11. O-O Nd7 12. a3 Be7 13. b4 <sep> Expanding his position very forcefully.\n',
 '13... Qe5 <sep> An error. Best was 13... Q - Q 1.\n',
 '14. f4 <sep> A stunning surprise. If 14... Q x K P ch, White plays 15 K - R 1 and the Black Queen is trapped. White will play 16 R - B 3 and win the errant Queen.\n',
 '14... Qb8 15. Bxf7+ <sep> The

In [6]:
new_output_path = os.path.join(output_dir_path, "game_commentary.txt")

In [7]:
with open(new_output_path, 'w') as f:
    newgames = 0
    for line in data:
        moves_list, _ = line.split("<sep>")
        if moves_list.startswith("1.") and not moves_list.startswith("1..."):
            newgames += 1
            f.write('\n\n')
        
        f.write(line.strip() + '<move>')

print(f"Total games found: {newgames}")
print(f"Data saved in: {new_output_path}")

Total games found: 11600
Data saved in: saved_files/game_commentary.txt
